# Métricas de rendimiento

## Indicadores de éxito

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Para contestar las preguntas necesitamos unir los dos dataframes
# Para ello, primero cargamos los dos dataframes
df = pd.read_csv('DATOS LIMPIOS/DATOS_LIMPIOS.txt')
df2 = pd.read_csv('DATOS LIMPIOS/proces_step_vist_id_data_time.txt')

In [4]:
# Unimos los dos dataframes
df = pd.merge(df, df2, on='client_id', how='inner')

In [6]:
df.head()

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,Variation,visitor_id,visit_id,process_step,date_time,time_diff,date,time_in_minutes,time_diff_seconds
0,836976,73.0,60.5,U,2.0,45105.3,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 10:59:43,0 days 00:01:08,2017-03-29,659,68.0
1,836976,73.0,60.5,U,2.0,45105.3,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 11:01:40,0 days 00:01:57,2017-03-29,661,117.0
2,836976,73.0,60.5,U,2.0,45105.3,Test,427070339_1413275162,104438405_2368283624_817211,start,2017-03-29 11:02:44,0 days 00:01:04,2017-03-29,662,64.0
3,836976,73.0,60.5,U,2.0,45105.3,Test,427070339_1413275162,228976764_46825473280_96584,step_1,2017-04-02 11:21:38,0 days 00:00:10,2017-04-02,681,10.0
4,836976,73.0,60.5,U,2.0,45105.3,Test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24,0 days 00:00:46,2017-04-02,682,46.0


In [7]:
# Guardamos el dataframe unido
df.to_csv('DATOS LIMPIOS/DATOS_LIMPIOS_UNIDOS.txt', index=False)

## Tasa de finalización :

La proporción de usuarios que llegan al paso final de "confirmación".

In [9]:
# Filtrar los datos por variación
df_test = df[df['Variation'] == 'Test']
df_control = df[df['Variation'] == 'Control']

# Calcular la tasa de finalización para cada grupo
tasa_finalizacion_test = df_test[df_test['process_step'] == 'confirm']['client_id'].nunique() / df_test['client_id'].nunique()
tasa_finalizacion_control = df_control[df_control['process_step'] == 'confirm']['client_id'].nunique() / df_control['client_id'].nunique()

print(f"Tasa de finalización (Test): {tasa_finalizacion_test:.2%}")
print(f"Tasa de finalización (Control): {tasa_finalizacion_control:.2%}")

Tasa de finalización (Test): 72.56%
Tasa de finalización (Control): 71.93%


Tasa de finalización (Test): 72.56%
Tasa de finalización (Control): 71.93%

## Tiempo dedicado a cada paso : 

la duración promedio que los usuarios dedican a cada paso.

In [ ]:
# Para realizar la sigiente métrica, tenemos que eliminar los outliers


In [22]:
# Eliminar outliers del dataframe
df_filtered = df[~df.index.isin(outliers.index)]

# Filtrar los datos por variación
df_test_filtered = df_filtered[df_filtered['Variation'] == 'Test']
df_control_filtered = df_filtered[df_filtered['Variation'] == 'Control']

# Calcular la duración promedio para cada paso en el grupo de test
duracion_promedio_test = df_test_filtered.groupby('process_step')['time_diff_seconds'].mean().reset_index()
duracion_promedio_test.columns = ['process_step', 'duracion_promedio_test']

# Calcular la duración promedio para cada paso en el grupo de control
duracion_promedio_control = df_control_filtered.groupby('process_step')['time_diff_seconds'].mean().reset_index()
duracion_promedio_control.columns = ['process_step', 'duracion_promedio_control']

# Unir los resultados en un solo dataframe
duracion_promedio_por_paso = pd.merge(duracion_promedio_test, duracion_promedio_control, on='process_step', how='outer')

print(duracion_promedio_por_paso)

  process_step  duracion_promedio_test  duracion_promedio_control
0      confirm               59.149954                  68.785423
1        start               53.189889                  40.123554
2       step_1               22.708585                  28.470278
3       step_2               33.687870                  31.407896
4       step_3               67.538846                  65.706207


In [31]:
# Comparar las duraciones promedio entre el grupo de control y el grupo de test
duracion_promedio_por_paso['mas_rapido'] = np.where(
    duracion_promedio_por_paso['duracion_promedio_test'] < duracion_promedio_por_paso['duracion_promedio_control'],
    'Test',
    'Control'
)

print(duracion_promedio_por_paso)

  process_step  duracion_promedio_test  duracion_promedio_control mas_rapido
0      confirm               59.149954                  68.785423       Test
1        start               53.189889                  40.123554    Control
2       step_1               22.708585                  28.470278       Test
3       step_2               33.687870                  31.407896    Control
4       step_3               67.538846                  65.706207    Control


process_step  duracion_promedio_test  duracion_promedio_control
0      confirm               59.149954                  68.785423
1        start               53.189889                  40.123554
2       step_1               22.708585                  28.470278
3       step_2               33.687870                  31.407896
4       step_3               67.538846                  65.706207

## Tasas de error :

Si hay un paso en el que los usuarios vuelven a un paso anterior, puede indicar confusión o un error. Debe considerar que pasar de un paso posterior a uno anterior es un error.

In [29]:
# Convertir la columna process_step a numérico categórico
step_mapping = {'start': 0, 'step_1': 1, 'step_2': 2, 'step_3': 3, 'confirm': 4}
df['process_step_num'] = df['process_step'].map(step_mapping)

# Calcular la tasa de error considerando los pasos hacia atrás como errores
df['previous_step_num'] = df.groupby('client_id')['process_step_num'].shift(1)
df['error'] = df['process_step_num'] < df['previous_step_num']

# Filtrar los datos por variación
df_test = df[df['Variation'] == 'Test']
df_control = df[df['Variation'] == 'Control']

# Calcular la tasa de error para cada grupo
tasa_error_test = df_test['error'].mean()
tasa_error_control = df_control['error'].mean()

print(f"Tasa de error (Test): {tasa_error_test:.2%}")
print(f"Tasa de error (Control): {tasa_error_control:.2%}")

Tasa de error (Test): 13.50%
Tasa de error (Control): 11.05%


Tasa de error (Test): 13.50%

Tasa de error (Control): 11.05%

## Resultado del rediseño

En función de los KPI elegidos, ¿cómo se compara el rendimiento del nuevo diseño con el anterior?

La tasa de finalización en el grupo de prueba (Test) es ligeramente superior a la del grupo de control. Esto indica que el nuevo diseño (Test) está logrando que un mayor porcentaje de usuarios complete la acción deseada en comparación con el diseño anterior (Control).

A pesar de que el grupo de prueba tiene una mayor tasa de finalización, el grupo de control es más rápido en completar cada paso del proceso. Esto sugiere que el nuevo diseño puede tener elementos que ralentizan a los usuarios o que requieren más tiempo para ser comprendidos.

La tasa de error es mayor en el grupo de prueba, lo que indica que los usuarios del nuevo diseño cometen más errores en comparación con los usuarios del diseño anterior. Esto podría estar relacionado con una interfaz menos intuitiva o con instrucciones que no son tan claras en el nuevo diseño.



